In [1]:
import pandas as pd

In [2]:
df_pos = pd.read_csv("Dataset_15_Mart_POS.csv")
df_item = pd.read_csv("Dataset_15_item_list.csv")

In [3]:
df_pos.head(0)

,Member_number,Date,itemDescription


In [4]:
df_item.head(0)

,prod_id,prod_nm,alcohol,frozen


In [9]:
df_date_cnt=df_pos["Date"].value_counts().reset_index()
# 기본적으로 내림차순 정렬이 된다. 
df_date_cnt = df_date_cnt.sort_values("Date", ascending = False).reset_index(drop=True)
df_date_cnt.head(10)

,index,Date
0,2015-01-21,96
1,2015-07-21,93
2,2015-08-08,92
3,2015-11-29,92
4,2015-04-30,91
5,2015-03-26,88
6,2015-09-30,87
7,2015-05-16,85
8,2015-06-19,85
9,2015-04-01,83


In [12]:
df_item_cnt = df_pos.loc[df_pos["Date"] == df_date_cnt.iloc[0, 0], "itemDescription"].value_counts()
df_item_cnt = df_item_cnt.reset_index()
df_item_cnt.head(1)

,index,itemDescription
0,soda,7


In [15]:
df_item_cnt.iloc[0, 1]

7

In [16]:
/---------------------------------------------------

TypeError: bad operand type for unary -: 'tuple'

In [17]:
df_pos.head(1)

,Member_number,Date,itemDescription
0,1220,2015-01-01,canned beer


In [23]:
df_pos["Date"] = pd.to_datetime(df_pos["Date"])
df_pos["month"] = df_pos["Date"].dt.month
df_pos["wday"] = df_pos["Date"].dt.weekday
df_pos["is_fs"] = (df_pos["wday"].isin([4, 5])) + 0 # 0: 월, 6: 일

In [24]:
pd.crosstab(df_pos["wday"], df_pos["is_fs"])

is_fs,0,1
wday,,
0,2789,0
1,2914,0
2,3021,0
3,2882,0
4,0,3002
5,0,2886
6,2994,0


In [25]:
df_item.head(2)

,prod_id,prod_nm,alcohol,frozen
0,1,tropical fruit,0,0
1,2,whole milk,0,0


In [29]:
df_join = df_pos.merge(df_item, 
                       left_on = "itemDescription",
                       right_on = "prod_nm",
                       how = "left")
df_join.head(2)

,Member_number,Date,itemDescription,month,wday,is_fs,prod_id,prod_nm,alcohol,frozen
0,1220,2015-01-01,canned beer,1,3,0,25,canned beer,1,0
1,1220,2015-01-01,margarine,1,3,0,70,margarine,0,0


In [47]:
df_join = df_join.loc[df_join["month"].isin(1,2,3),]

TypeError: isin() takes 2 positional arguments but 4 were given

In [48]:
df_join_sub = df_join.loc[df_join["alcohol"] == 1,]

In [49]:
df_agg = df_join.groupby(["Date", "is_fs"])["alcohol"].sum().reset_index()


In [50]:
df_agg.head(10)

,Date,is_fs,alcohol
0,2015-01-01,0,4
1,2015-01-02,1,3
2,2015-01-03,1,2
3,2015-01-04,0,4
4,2015-01-05,0,5
5,2015-01-06,0,4
6,2015-01-07,0,5
7,2015-01-08,0,1
8,2015-01-09,1,3
9,2015-01-10,1,2


In [38]:
from scipy.stats import ttest_ind

In [74]:
stat, p = ttest_ind(df_agg.loc[df_agg["is_fs"] ==0, "alcohol"],
                    df_agg.loc[df_agg["is_fs"] == 1, "alcohol"])           
round(p, 2)

0.94

In [ ]:
/---------------------------------------------------------------------

In [51]:
df_pos.head(1)

,Member_number,Date,itemDescription,month,wday,is_fs
0,1220,2015-01-01,canned beer,1,3,0


In [53]:
df_item_cnt = df_pos["itemDescription"].value_counts()

In [54]:
df_item_cnt[:10]

whole milk          1464
other vegetables    1039
rolls/buns           879
sausage              744
soda                 737
yogurt               694
tropical fruit       668
root vegetables      592
citrus fruit         544
frankfurter          505
Name: itemDescription, dtype: int64

In [55]:
df_pos["top_10"] = df_pos["itemDescription"].isin(df_item_cnt[:10].index) +0

In [57]:
df_pos.head(10)

,Member_number,Date,itemDescription,month,wday,is_fs,top_10
0,1220,2015-01-01,canned beer,1,3,0,0
1,1220,2015-01-01,margarine,1,3,0,0
2,1220,2015-01-01,chocolate,1,3,0,0
3,1235,2015-01-01,sausage,1,3,0,1
4,1235,2015-01-01,sausage,1,3,0,1
5,1235,2015-01-01,bottled beer,1,3,0,0
6,1235,2015-01-01,spread cheese,1,3,0,0
7,1422,2015-01-01,tropical fruit,1,3,0,1
8,1422,2015-01-01,turkey,1,3,0,0
9,1422,2015-01-01,salty snack,1,3,0,0


In [69]:
df_pos_agg = df_pos.groupby(["Date", "wday"])["top_10"].sum().reset_index()

In [70]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [71]:
model = ols("top_10 ~ C(wday)", data = df_pos_agg).fit()

In [72]:
model_report = anova_lm(model)

In [73]:
round(model_report["PR(>F)"][0],2)

0.52